## Word2vec for movie reviews

Build word2vec model using tensorflow, with movie review data (5331 positive and 5331 negative snippets) from http://www.cs.cornell.edu/people/pabo/movie-review-data

### Clean data

In [26]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize

file_pos = open("rt-polarity-pos.txt", "r", encoding="ISO-8859-1")
data_pos = file_pos.readlines()
data_pos[:10]
file_neg = open("rt-polarity-neg.txt", "r", encoding="ISO-8859-1")
data_neg = file_neg.readlines()
data_neg[:10]

['simplistic , silly and tedious . \n',
 "it's so laddish and juvenile , only teenage boys could possibly find it funny . \n",
 'exploitative and largely devoid of the depth or sophistication that would make watching such a graphic treatment of the crimes bearable . \n',
 '[garbus] discards the potential for pathological study , exhuming instead , the skewed melodrama of the circumstantial situation . \n',
 'a visually flashy but narratively opaque and emotionally vapid exercise in style and mystification . \n',
 "the story is also as unoriginal as they come , already having been recycled more times than i'd care to count . \n",
 "about the only thing to give the movie points for is bravado -- to take an entirely stale concept and push it through the audience's meat grinder one more time . \n",
 'not so much farcical as sour . \n',
 'unfortunately the story and the actors are served with a hack script . \n',
 'all the more disquieting for its relatively gore-free allusions to the seria

In [39]:
df = pd.DataFrame([data_neg, data_pos]).T
df

,0,1
0,"simplistic , silly and tedious . \n",the rock is destined to be the 21st century's ...
1,"it's so laddish and juvenile , only teenage bo...","the gorgeously elaborate continuation of "" the..."
2,exploitative and largely devoid of the depth o...,effective but too-tepid biopic\n
3,[garbus] discards the potential for pathologic...,if you sometimes like to go to the movies to h...
4,a visually flashy but narratively opaque and e...,"emerges as something rare , an issue movie tha..."
5,"the story is also as unoriginal as they come ,...",the film provides some great insight into the ...
6,about the only thing to give the movie points ...,offers that rare combination of entertainment ...
7,not so much farcical as sour . \n,perhaps no picture ever made has more literall...
8,unfortunately the story and the actors are ser...,steers turns in a snappy screenplay that curls...
9,all the more disquieting for its relatively go...,take care of my cat offers a refreshingly diff...


In [67]:
#nltk.download('stopwords')
STOP_WORDS = nltk.corpus.stopwords.words()

def clean_sentence(text):
    text=text.lower()
    text = re.sub('@[^\s]+','', text)
    text = re.sub('#([^\s]+)', '', text)
    text = re.sub('[:;>?<=*+()&,\-#!$%\{˜|\}\[^_\\@\]1234567890’‘]',' ', text)
    text = re.sub('[\d]','', text)
    text = text.replace(".", '')
    #text = text.replace("'", '')
    text = text.replace("`", '')
    text = text.replace("'s", '')
    text = text.replace("/", ' ')
    text = text.replace("\"", ' ')
    text = text.replace("\\", '')
    text=re.sub( '\s+', ' ', text).strip()
    
    words = text.split(" ")
    for word in list(words):
        if word in STOP_WORDS:
            words.remove(word)
        if word == "":
            words.remove(word)

    return words

corpus = []
for sentence in df[:][0]:
    corpus.append(clean_sentence(sentence))
    
print(corpus[:5])

#print(data)
#print(df[0][6])
#print(data)
#print(corpus)


[['simplistic', 'silly', 'tedious'], ['laddish', 'juvenile', 'teenage', 'boys', 'could', 'possibly', 'find', 'funny'], ['exploitative', 'largely', 'devoid', 'depth', 'sophistication', 'would', 'make', 'watching', 'graphic', 'treatment', 'crimes', 'bearable'], ['garbus', 'discards', 'potential', 'pathological', 'study', 'exhuming', 'instead', 'skewed', 'melodrama', 'circumstantial', 'situation'], ['visually', 'flashy', 'narratively', 'opaque', 'emotionally', 'vapid', 'exercise', 'style', 'mystification']]


In [56]:
from nltk.tokenize import word_tokenize
text = "I love programming and programming also loves me"
tokens=word_tokenize(text)

LookupError: 
**********************************************************************
  Resource 'tokenizers/punkt/PY3/english.pickle' not found.
  Please use the NLTK Downloader to obtain the resource:  >>>
  nltk.download()
  Searched in:
    - '/Users/qiuruihao/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************